# Outline:

1. welcome message
2. chat about user objectives
3. upload resume
4. assess resume: skills, experience
5. chat about industry/job
6. load current jobs CSV
7. suggest plan
8. find relevant courses
9. suggest outreach plan
10. make new resume

In [ ]:
#Install Google Auth API access
!pip install --q google-auth-oauthlib;
!pip install --q google-auth;
!pip install --upgrade -q google-auth;

#Install Langchain
!pip install --q langchain openai chromadb;
!pip install --q tiktoken;
!pip install --q langchain-experimental;
!pip install --upgrade --quiet  langchain-google-genai pillow;
!pip install langchain_openai --q

!pip install PyPDF2 --q;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.29.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.4 MB/s 

In [ ]:
!pip install networkx matplotlib --q

In [ ]:
import getpass
import os

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("""Welcome to career coaching with Career Gemini.
    Career Gemini is a helpful agent to help you plan your career. Make the text shorter than 350 characters
    Make it sound helpful and intelligent. Ask the user to share the purpose of today's session.
    Make recommendation on suggested promtps like "plan a career pivot", "find a new job that fits my current profile" or
    "plan a long-term career advancement plan". """)
to_markdown(response.text)

> Welcome to Career Gemini, your career coaching assistant.
> 
> Let's start by understanding your goals today. Are you looking to:
> 
> * Plan a career pivot?
> * Find a new job aligned with your current profile?
> * Create a long-term career advancement plan?
> 
> Your objectives will guide our session. Together, we'll navigate your career journey effectively.

In [ ]:
class CareerChatBot:
    def __init__(self):
        self.user_data = {'career_goals': ''}  # Initialize as an empty string

    def send_message(self, message, stream=False):
        print(message)
        if stream:
            return self.stream_response()
        else:
            return iter([input("Enter your goal or type 'done' to finish: ")])

    def stream_response(self):
        while True:
            user_input = input("Enter another goal or type 'done' to finish: ")
            if user_input.lower() == 'done':
                break
            yield user_input

    def get_chat_response(self, chat, prompt):
        text_response = []
        responses = chat.send_message(prompt, stream=True)
        for chunk in responses:
            text_response.append(chunk)
        return ''.join(text_response)

    def collect_career_goals(self):
        # Start a chat session and ask for career goals
        prompt = "Hi! I'm your Career Goal Bot. Let's talk about your career goals. " \
                 "Tell me one career goal at a time. Type 'done' when you are finished."
        user_goals = self.get_chat_response(self, prompt)

        # Split the collected goals into a list and store them as a comma-separated string
        goals_list = [goal.strip() for goal in user_goals.split('\n') if goal.strip()]
        self.user_data['career_goals'] = ', '.join(goals_list)

    def get_career_goals(self):
        # Retrieve and return career goals as a string
        return self.user_data['career_goals']

    def display_goals(self):
        # Display the collected career goals
        if self.user_data['career_goals']:
            prompt = f"""Describe in expanded form the user goals isted in {self.user_data['career_goals']}.
            You are a compassionate career coach that has been listening to the user describe their career objectives,
            start with the phrase \"Your career goals are:\, include each one of the words used in the prompt,
            make sure the response invites further understanding of the user background"""
            response = model.generate_content(prompt)
            print(response.text)
        else:
            print("No career goals recorded.")

# Main interaction
if __name__ == "__main__":
    bot = CareerChatBot()
    bot.collect_career_goals()
    bot.display_goals()


Hi! I'm your Career Goal Bot. Let's talk about your career goals. Tell me one career goal at a time. Type 'done' when you are finished.
Enter another goal or type 'done' to finish: grow in my caree
Enter another goal or type 'done' to finish: develop new skills
Enter another goal or type 'done' to finish: done
Your career goals are:

- **Grow in my career:** This could mean advancing to a higher position, taking on new responsibilities, becoming a leader, or making a significant contribution to your field.

- **Develop new skills:** This could include acquiring technical skills, soft skills, or knowledge in a new area. By expanding your skill set, you can increase your employability and make yourself more valuable to potential employers.

I would love to hear more about your background and experience. This will help me to better understand your goals and how I can support you in achieving them.


# Upload Resume

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving EliaBrodsky Resume - 010124.pdf to EliaBrodsky Resume - 010124.pdf


# Resume Parsing

In [ ]:
import PyPDF2
import pandas as pd
from IPython.display import Markdown

# Assuming all your imports for langchain and Google Generative AI work as intended

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        document_text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            if page_text:
                document_text += page_text

    return document_text

def chat_with_pdf_and_store_data(pdf_path):
    global response_df

    document_text = extract_text_from_pdf(pdf_path)

    questions = [
        "What is the user name, email, address and linkedin?",
        "Summarize user background in 3 sentences",
        "What are the jobs the user held with start date, end date and key experience in each position?",
        "What is the user background in terms of industry, length of experience?",
        "How many years of experience does the user have in which industries?",
        "What are the specific skills the user brings to the table?"
    ]

    responses = []  # Initialize list to store responses

    for question in questions:
        response = query_gemini(question, document_text)  # Pass document context
        responses.append(response)  # Store response in the list

    # Create a DataFrame to display questions and responses
    data = {"Question": questions, "Response": [response.text for response in responses]}
    response_df = pd.DataFrame(data)

    # Display the DataFrame in Markdown format in the notebook
    display(Markdown(response_df.to_markdown()))

def query_gemini(question, document_context):
    # Create an instance of the Gemini LLM
    llm = genai.GenerativeModel('gemini-pro')

    # Combine user prompt and document context
    combined_input = f"{question} {document_context}"

    # Invoke the model with the combined input
    result = llm.generate_content(combined_input)

    # Extract and return the response from the result
    response = result
    return response

pdf_path = "/content/EliaBrodsky Resume - 010124.pdf"
chat_with_pdf_and_store_data(pdf_path)

|    | Question                                                                                       | Response                                                                                                                                                                                                                                                                                                                                                                                             |
|---:|:-----------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | What is the user name, email, address and linkedin?                                            | **User Name:** Eliabrodsky                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **Email:** ebrodsky@tulane.edu                                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **Address:** New Orleans, LA                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **LinkedIn:** linkedin.com/in/eliabrodsky                                                                                                                                                                                                                                                                                                                                                            |
|  1 | Summarize user background in 3 sentences                                                       | Eli Brodsky is an executive professional with an MBA and substantial experience in information technologies, marketing, and healthcare strategy. He holds a proven track record of successful execution, innovation implementation, and change management, with a commitment to data-driven decision-making for operational excellence.                                                              |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | Brodsky's notable expertise spans strategy and planning, business development, digital marketing, financial analysis, management, and technology implementation. He has a deep understanding of healthcare and life sciences data management, bioinformatics, and healthcare analytics, as well as expertise in software development processes, healthcare credentialing, and workforce development. |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | Throughout his career, he has held various leadership positions in organizations ranging from startups to large healthcare providers, including DePaul Community Health Centers, Pine Biotech, Inc., and Georgetown University School of Medicine.                                                                                                                                                   |
|  2 | What are the jobs the user held with start date, end date and key experience in each position? | **1. ASCENSION HEALTH - DEPAUL COMMUNITY HEALTH CENTERS**                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * Business Development Executive, (December 2022 - Present, New Orleans, LA)                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                |     * Orchestrated streamlined operations across 11 clinics and 26 schools, integrating value-based contracts and metrics for healthcare providers.                                                                                                                                                                                                                                                  |
|    |                                                                                                |     * Collaborated with the executive team to assess strategic opportunities, develop project plans, and report on key performance indicators (KPIs) with a focus on Medicare and Medicaid standards.                                                                                                                                                                                                |
|    |                                                                                                |     * Led the implementation of data governance initiatives to fortify data security, ensuring accurate reporting and compliance with evolving standards.                                                                                                                                                                                                                                            |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **2. PINE BIOTECH, INC.**                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                | * Co-founder & CEO, (2014 - 2022, New Orleans, LA, Boston, MA)                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                |     * Raised over $1 million in startup capital from private and corporate investors.                                                                                                                                                                                                                                                                                                                |
|    |                                                                                                |     * Secured federal agency grants (USDA, NIH, DoD/DARPA, NSF) for life-science data analytics and educational business development.                                                                                                                                                                                                                                                                |
|    |                                                                                                |     * Developed compelling customer case studies, financial projections, and product presentations.                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                |     * Achieved profitability with an annual growth rate exceeding 150% YoY.                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Managed project proposals with budgets exceeding $1 million, including federal grants and academic grant subcontracts.                                                                                                                                                                                                                                                                         |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **3. METHENA**                                                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                | * Product Development Advisor, (2022 - 2023, San Francisco, CA)                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                |     * Advised the CEO and founder, playing a pivotal role in strategic planning and Series A fundraising preparation.                                                                                                                                                                                                                                                                                |
|    |                                                                                                |     * Innovated by introducing solutions in sentiment analysis, natural language processing (NLP), and developing a predictive model using proprietary and public domain higher education data.                                                                                                                                                                                                      |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **4. GEORGETOWN UNIVERSITY SCHOOL OF MEDICINE**                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * Course Author & Instructor, (2019 - 2022, Washington D.C.)                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                |     * Authored and instructed the graduate Systems Medicine Program course, "Machine Learning for Biomedical Data"                                                                                                                                                                                                                                                                                   |
|    |                                                                                                |     * Developed a project-based 3-credit course emphasizing workforce-ready skills in data analytics, Python programming, and their applications in medical and life science research.                                                                                                                                                                                                               |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **5. OTHER EXPERIENCE AND ACCOMPLISHMENTS:**                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * THE IRON NETWORK LLC (OWNER, DIGITAL MARKETING EXPERT) 2012 - 2015 GULFPORT, MS)                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                |     * Founded and led a successful digital marketing agency focusing on technology startups and small to medium-sized businesses.                                                                                                                                                                                                                                                                    |
|    |                                                                                                |     * Expanded the team to over 20 members, overseeing marketing, software engineering, and business development departments.                                                                                                                                                                                                                                                                        |
|    |                                                                                                |     * Established profitable partnerships with marketing agencies across multiple states.                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * VIDADBOX  (CO-FOUNDER & PARTNER) 2008 - 2014 HAIFA, ISRAEL)                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                |     * Co-founded and managed a video marketing animation studio specializing in video production for television and digital advertising.                                                                                                                                                                                                                                                             |
|    |                                                                                                |     * Developed initial business strategy, marketing, and sales efforts.                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                |     * Led a team of animators and video professionals for diverse projects.                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Directed a weekly television news show in Jerusalem and provided video editing services to international TV channels.                                                                                                                                                                                                                                                                          |
|  3 | What is the user background in terms of industry, length of experience?                        | **Industry Experience:**                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * Information Technology                                                                                                                                                                                                                                                                                                                                                                             |
|    |                                                                                                | * Marketing                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                | * Healthcare Strategy                                                                                                                                                                                                                                                                                                                                                                                |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | **Length of Experience:**                                                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | * Over 10 years of executive-level experience in the healthcare, life sciences, education, and technology industries                                                                                                                                                                                                                                                                                 |
|  4 | How many years of experience does the user have in which industries?                           | Eli Brodsky has experience in the following industries:                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                |                                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | - Information technologies                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                | - Marketing                                                                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                | - Healthcare strategy                                                                                                                                                                                                                                                                                                                                                                                |
|    |                                                                                                | - Life sciences                                                                                                                                                                                                                                                                                                                                                                                      |
|    |                                                                                                | - Data management                                                                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                | - Bioinformatics                                                                                                                                                                                                                                                                                                                                                                                     |
|    |                                                                                                | - Healthcare analytics                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                | - Software development                                                                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                | - Healthcare credentialing                                                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                | - Onboarding                                                                                                                                                                                                                                                                                                                                                                                         |
|    |                                                                                                | - Workforce development                                                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                | - Recruitment                                                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                | - Training                                                                                                                                                                                                                                                                                                                                                                                           |
|  5 | What are the specific skills the user brings to the table?                                     | * **Strategy, Planning, Risk, and Project Management:**                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                |     * Develops and executes strategic initiatives and plans                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Manages risk and projects to ensure successful implementation                                                                                                                                                                                                                                                                                                                                  |
|    |                                                                                                | * **Business Development, Digital Marketing, and Sales Support:**                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                |     * Generates new business opportunities through effective marketing and sales strategies                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Provides support to sales teams to increase their productivity                                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                | * **Financial Analysis, Estimates, and Projections:**                                                                                                                                                                                                                                                                                                                                                |
|    |                                                                                                |     * Analyzes financial data to make informed business decisions                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                |     * Creates and presents financial estimates and projections                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                | * **Management, Process Optimization, and Operational Effectiveness:**                                                                                                                                                                                                                                                                                                                               |
|    |                                                                                                |     * Optimizes processes to improve operational efficiency                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Manages teams and resources effectively                                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                | * **Implementation of technology and software solutions:**                                                                                                                                                                                                                                                                                                                                           |
|    |                                                                                                |     * Evaluates, selects, and implements technology solutions to meet business needs                                                                                                                                                                                                                                                                                                                 |
|    |                                                                                                |     * Oversees the development and deployment of software systems                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                | * **Healthcare and Life Sciences Data Management, Bioinformatics, and Healthcare Analytics:**                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                |     * Manages healthcare and life sciences data effectively                                                                                                                                                                                                                                                                                                                                          |
|    |                                                                                                |     * Applies bioinformatics and healthcare analytics to extract insights and make data-driven decisions                                                                                                                                                                                                                                                                                             |
|    |                                                                                                | * **Management of Software Development Processes and Teams:**                                                                                                                                                                                                                                                                                                                                        |
|    |                                                                                                |     * Manages software development processes and teams to ensure efficient and effective project completion                                                                                                                                                                                                                                                                                          |
|    |                                                                                                | * **Healthcare Credentialing and Onboarding:**                                                                                                                                                                                                                                                                                                                                                       |
|    |                                                                                                |     * Ensures compliance with healthcare regulations and standards                                                                                                                                                                                                                                                                                                                                   |
|    |                                                                                                |     * Onboards new healthcare providers and staff                                                                                                                                                                                                                                                                                                                                                    |
|    |                                                                                                | * **Workforce Development, Recruitment, and Training:**                                                                                                                                                                                                                                                                                                                                              |
|    |                                                                                                |     * Develops and implements workforce training programs                                                                                                                                                                                                                                                                                                                                            |
|    |                                                                                                |     * Recruits and trains employees to meet organizational needs                                                                                                                                                                                                                                                                                                                                     |

In [ ]:
# Combine user prompt and document context
question = """Read the document and summarize the uploaded resume based on your objectives
  as a career coach who is speaking with a motivated person who is trying to achieve their goals,
  look at the important parts of the uploaded document to determine and recommend an evaluation,
  suggesting next step to extract skills from the resume.
  End with "would you like to proceed?"""
combined_input = f"{question} {response_df}"

# Invoke the model with the combined input
result = model.generate_content(combined_input)
to_markdown(result.text)

> **Summary:**
> 
> Eli Brodsky is a motivated and experienced executive professional with a strong background in information technology, strategy, planning, risk management, and project management. He has held senior leadership roles in the healthcare and financial services industries for over 15 years. Eli is a highly analytical and results-oriented leader with a proven track record of success in driving organizational growth and profitability.
> 
> **Evaluation:**
> 
> Eli's resume highlights his extensive experience and qualifications in his field. His strong leadership skills, coupled with his deep understanding of business operations and technology, make him a valuable asset to any organization.
> 
> **Next Steps:**
> 
> To further assess Eli's skills and potential, I recommend the following next steps:
> 
> * Conduct a skills extraction interview to identify his specific technical and soft skills.
> * Review his portfolio of work or ask him to provide examples of his accomplishments.
> 
> **Would you like to proceed with these steps?**

# Resume Parsing to Skill

In [ ]:
import PyPDF2
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown

# Create a single instance of the model to be reused
model = genai.GenerativeModel('gemini-pro')

def extract_text_from_pdf(pdf_path):
    """Extracts text from each page of the given PDF and returns the complete text."""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return ''.join(page.extract_text() for page in pdf_reader.pages if page.extract_text())

def create_dataframe_from_pdf(document_text):
    """Converts document text to a DataFrame by parsing lines separated by commas."""
    data = [line.split(',') for line in document_text.split('\n') if line.strip()]
    if not data:
        return pd.DataFrame()
    return pd.DataFrame(data[1:], columns=data[0]) if all(len(row) == len(data[0]) for row in data) else pd.DataFrame(data)

def generate_recommendations(document_text):

    prompt = """Assess user skills and experience based on provided document text,
    define the following 5 columns: 'Skill', 'Significance', 'Mastery Level', 'Recommendation', 'Time',
    use short names and concise langauge, do not use any unusual characters (i.e. \n, &, %, etc.),
    for skills, use specific skills that can be developed, avoid generic terms and industry titles,
    determine which skills are most critical to improve from 0 to 100,
    determine user skill level judging by experience in the provided document,
    after analysis, output results as follows:
    1. in column "Skill", list skills in descending order, using no more than 2 words per skill name,
    2. add column "Significance" and write level of significance int hat column from 0 to 100,
    3. in column "Mastery Level", provide an assessment to the user on skill level from 0 to 100,
    4. in column 4, provide recommendations for each skill on improvement,
    5. in column 5, provide estimated time (in months) to complete the recommendation,
    make recommendations specific, using such things as taking a course (specific course title),
    developing a project (project title suggestion), networking with internal stakeholders,
    demonstrating skills in a project, etc.,
    format the output as a table"""
    combined_input = f"{prompt} {document_text}"
    return model.generate_content(combined_input).text

def read_and_analyze_pdf(pdf_path):

    global parsed_dataframe
    global recommendations

    """Reads PDF, extracts text, creates a DataFrame, and generates recommendations."""
    document_text = extract_text_from_pdf(pdf_path)
    parsed_dataframe = create_dataframe_from_pdf(document_text)
    recommendations = generate_recommendations(document_text)
    return recommendations

####
pdf_path = "/content/EliaBrodsky Resume - 010124.pdf"
output = read_and_analyze_pdf(pdf_path)
to_markdown(output)

> | Skill | Significance | Mastery Level | Recommendation | Time |
> |---|---|---|---|---|
> | Machine Learning | 100 | 90 | Take a course on machine learning offered by platforms like Coursera | 2 |
> | Data Analytics | 100 | 80 | Develop a project to apply data analytics techniques to a real-world dataset | 3 |
> | Project Management | 90 | 70 | Obtain a certification in project management, such as the PMP from PMI | 4 |
> | Leadership | 80 | 60 | Network with internal stakeholders to gain insights and build relationships | 2 |
> | Strategy Development | 70 | 70 | Take a course on strategy development offered by top universities | 1 |

In [ ]:
# Define your input data as a multi-line string.
data = recommendations
# Split the data into rows, then further split by '|'
rows = data.strip().split('\n')
data_parsed = [row.split('|') for row in rows]

# Create DataFrame
df = pd.DataFrame(data_parsed)


# Set the first column as the index
df.set_index(df.columns[0], inplace=True)

# Set the first row as column names
df.columns = df.iloc[0]
df.columns = [col.replace(" ", "").replace("*", "") for col in df.columns]
df['Significance'] = pd.to_numeric(df['Significance'], errors='coerce')
df['MasteryLevel'] = pd.to_numeric(df['MasteryLevel'], errors='coerce')


df = df[2:]


# Display the DataFrame to confirm
df


,Skill,Significance,MasteryLevel,Recommendation,Time,
0,,,,,,
,Machine Learning,100.0,90.0,Take a course on machine learning offered by ...,2,
,Data Analytics,100.0,80.0,Develop a project to apply data analytics tec...,3,
,Project Management,90.0,70.0,"Obtain a certification in project management,...",4,
,Leadership,80.0,60.0,Network with internal stakeholders to gain in...,2,
,Strategy Development,70.0,70.0,Take a course on strategy development offered...,1,


In [ ]:
import plotly.graph_objects as go

categories = df['Skill']

fig = go.Figure()

fig.add_traces(go.Scatterpolar(
    r = df['Significance'] ,
    theta=categories,
    fill='toself',
    name='Significance'
))

fig.add_traces(go.Scatterpolar(
    r= df['MasteryLevel'] ,
    theta=categories,
    fill='toself',
    name='Mastery Level'
))

fig.update_layout(
    width=800,
    polar=dict(
        radialaxis=dict(
            visible=False,
            range=[0,100]
        )
    ),
    showlegend=True
)

fig.show()



In [ ]:
!pip install plotly --q

In [ ]:
df

,Skill,Significance,MasteryLevel,Recommendation,Time,
0,,,,,,
,Data Analysis,75.0,85.0,Take a more advanced course in machine learni...,6,
,Project Management,90.0,80.0,"Get certified in project management (e.g., PM...",3,
,Business Development,80.0,75.0,Network with internal stakeholders and develo...,2,
,Financial Management,70.0,60.0,Take a course in financial management or acco...,4,
,Software Development,50.0,50.0,Develop a project that demonstrates your skil...,5,


In [ ]:
import pandas as pd
import plotly.express as px

df['ScaledMasteryLevel'] = round(df['MasteryLevel'] ** 1.2,0)  # Adjust this exponent as needed

# Create the bubble chart
fig = px.scatter(df,
                 x='Time',
                 y='Significance',
                 size='ScaledMasteryLevel',  # Bubble size
                 color='Skill',  # Differentiating data points by skill for visual distinction
                 text='Skill',
                 hover_name='Skill',  # Display skill names on hover
                 size_max=40,
                 title='Significance vs Time to Complete with Current Mastery Level as Size')

# Customize text position and size
fig.update_traces(textposition='top center',
                  textfont_size=10)  # Reduce font size here

# Customize the layout
fig.update_layout(width = 1000,
                  height = 800,
                  xaxis_title='Time to Complete (months)',
                  yaxis_title='Significance',
                  legend_title='Skills')

# Show the plot
fig.show()


In [ ]:
months_selected = input("Specify your time committment estimate (in months):")

prompt = """
        You are a career coach,
        the user selects the number of months they want to commit to improvement (months_selected),
        using the provided text (output), explain in detail how the user should prioritize
        their skill development by choosing the skill that they can improve the most in the
        time selected for time commitment (months_selected), providing the user with estimated time (in months)
        to improve the level of mastery of the sselected skill, estimating the impact of improvement.
        Be sure to add specific recommendations on how to achive desirted outcome framing the response
        as advancement in the career of choice. The user will expect time allocation to match their time commitment in months,
        specific SMART goals (specific, measurable, attainable, realistic and timely) and results that
        they can use to improve the resume"""
combined_input = f"{prompt} user preferred timeframe is: {months_selected}, user assessment is {output}"
to_markdown(model.generate_content(combined_input).text)

Specify your time committment estimate (in months):12


> **Prioritizing Skill Development**
> 
> Based on your selection of 12 months for your improvement commitment (months_selected), the optimal prioritization strategy is to focus on the skills that offer the greatest potential for growth within that timeframe. Let's analyze the provided text:
> 
> * **Machine Learning:** While Machine Learning holds the highest significance, your current mastery level is already at 90, indicating that substantial improvement may require more time.
> 
> * **Data Analytics:** With a mastery level of 80, Data Analytics offers a promising opportunity for growth. Aim to dedicate 3 months to a real-world project that showcases your analytical abilities.
> **SMART Goal:** Enhance mastery level in Data Analytics from 80% to 90% within 3 months.
> 
> * **Project Management:** Improving your Project Management skills requires obtaining a certification, such as the PMP from PMI. While this certification can enhance your credibility, it may not yield significant results within the 12-month timeframe.
> 
> * **Leadership:** Your current mastery level of 60 indicates significant room for improvement. However, to achieve substantial growth, you will need additional time beyond the 12 months.
> 
> * **Strategy Development:** With a mastery level of 70, Strategy Development may not offer the most significant growth potential within the 12-month timeframe.
> 
> **Recommendations for Achieving Desired Outcomes**
> 
> * **Allocate Time Effectively:** Dedicate 3 months to developing Data Analytics skills through a real-world project.
> * **SMART Goal Alignment:** Measure your progress against the target of improving Data Analytics mastery from 80% to 90% within 3 months.
> * **Maximize Impact:** Showcase your enhanced Data Analytics skills on your resume, demonstrating real-world problem-solving capabilities and increasing your competitiveness in the job market.
> 
> By prioritizing Data Analytics development within the 12-month timeframe while setting specific, measurable, and attainable goals, you can advance your career in your chosen field.

In [ ]:
months_selected = input("Specify your time committment estimate (in months):")

prompt = """
        You are a career coach,
        the user selects the number of months they want to commit to improvement (months_selected),
        using the provided text (output), explain in detail how the user should prioritize
        their skill development by choosing the skill that they can improve the most in the
        time selected for time commitment (months_selected), providing the user with estimated time (in months)
        to improve the level of mastery of the sselected skill, estimating the impact of improvement.
        Be sure to add specific recommendations on how to achive desirted outcome framing the response
        as advancement in the career of choice. The user will expect time allocation to match their time commitment in months,
        specific SMART goals (specific, measurable, attainable, realistic and timely) and results that
        they can use to improve the resume"""
combined_input = f"{prompt} user preferred timeframe is: {months_selected}, user assessment is {output}"
output = model.generate_content(combined_input).text
to_markdown(output)

Specify your time committment estimate (in months):3


> **Prioritizing Skill Development for Career Advancement**
> 
> To maximize your career advancement in the next 3 months, prioritize skill development based on the following criteria:
> 
> **1. Time to Improve:**
> 
> Consider skills where you can achieve significant improvement within the 3-month timeframe. Focus on skills with shorter estimated timeframes.
> 
> **2. Impact on Career Goals:**
> 
> Select skills that align with your desired career path. The more relevant the skills, the greater the impact they will have on your professional growth.
> 
> **3. Mastery Level:**
> 
> Aim to focus on skills where you have a strong foundation (at least 80% mastery). This will allow you to build upon your existing knowledge quickly.
> 
> **Based on the provided user assessment, the following skills meet these criteria:**
> 
> **Recommended Skill:** **Data Analytics**
> 
> * **Significance:** 100
> * **Mastery Level:** 80
> * **Estimated Time to Improve:** 3 months
> 
> **SMART Goal:**
> 
> * Within 3 months, develop a project to apply data analytics techniques to a real-world dataset, increasing mastery level to 95%.
> 
> **Action Plan:**
> 
> * Dedicate 3-4 hours per week to project development.
> * Leverage online resources and seek guidance from mentors or peers.
> * Use tools like Python and SQL for data manipulation and analysis.
> 
> **Estimated Impact:**
> 
> * Improved resume with tangible evidence of data analytics expertise.
> * Increased confidence in handling data-driven projects and decision-making.
> * Potential for promotion or lateral move to a data-driven role.
> 
> By focusing on skills that can be significantly improved within your chosen timeframe and aligning with your career goals, you can optimize your skill development strategy and achieve substantial career advancement.

In [ ]:
output

'**Prioritizing Skill Development for Career Advancement**\n\nTo maximize your career advancement in the next 3 months, prioritize skill development based on the following criteria:\n\n**1. Time to Improve:**\n\nConsider skills where you can achieve significant improvement within the 3-month timeframe. Focus on skills with shorter estimated timeframes.\n\n**2. Impact on Career Goals:**\n\nSelect skills that align with your desired career path. The more relevant the skills, the greater the impact they will have on your professional growth.\n\n**3. Mastery Level:**\n\nAim to focus on skills where you have a strong foundation (at least 80% mastery). This will allow you to build upon your existing knowledge quickly.\n\n**Based on the provided user assessment, the following skills meet these criteria:**\n\n**Recommended Skill:** **Data Analytics**\n\n* **Significance:** 100\n* **Mastery Level:** 80\n* **Estimated Time to Improve:** 3 months\n\n**SMART Goal:**\n\n* Within 3 months, develop a